In [1]:
#Packages
import pandas as pd
import requests
from bs4 import BeautifulSoup
import lxml
import random
import time
import re
import subprocess
import threading

In [ ]:

servers = ['Algeria','Egypt','Ghana',"Argentina", "Bahamas", "Bermuda", "Bolivia", "Brazil", "Brazil - 2",'Kenya','Israel','Morocco','Lebanon',
           'South Africa','Albania','Andorra','Armenia']

def rotate_vpn(servers):
    while True:
        try:
            random_server = random.choice(servers)
            subprocess.run(['expressvpn', 'disconnect'], check=True)
            subprocess.run(['expressvpn', 'connect', random_server], check=True)
            print(f"Connected to {random_server}")
            time.sleep(5)  # Give the VPN time to stabilize
        except subprocess.CalledProcessError as e:
            print(f"Failed to switch VPN server: {e}")
        except KeyboardInterrupt:
            print("Stopping VPN rotation.")
            break
        time.sleep(100) 

# Start the VPN rotation in a separate thread
stop_event = threading.Event()
vpn_thread = threading.Thread(target=rotate_vpn, args=(servers,))
vpn_thread.start()

baseurl = "https://themanifest.com"
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.6533.103 Mobile Safari/537.36",
    "Mozilla/5.0 (iPod; CPU iPhone OS 17_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/127.0.6533.107 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (iPad; CPU OS 17_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/127.0.6533.107 Mobile/15E148 Safari/604.1"
    # Add more user agents here
]
user_agent = random.choice(USER_AGENTS)
headers = {
    'User-Agent': user_agent
}
baseurl = "https://themanifest.com"
agencylinks = []

for x in range(1):
    url = f"https://themanifest.com/ke/advertising/agencies?page={x}"
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.content, "lxml")

    # Find all <a> tags with the class "provider-card__profile-link"
    profile_links = soup.find_all('a', class_='provider-card__profile-link')
    
    # Extract the href attributes and concatenate with baseurl
    urls = [baseurl + link['href'] for link in profile_links]
    
    # Store the concatenated URLs in agencylinks
    agencylinks.extend(urls)


    # Code to get all the data from each link
    agency_list = []
    for link in agencylinks:
        r = requests.get(link, headers=headers)
        soup = BeautifulSoup(r.content, "lxml")
    
        # Initialize variables with default values
        agency_name = "Not available"
        website_link = "Not available"
        date_started = "Not available"
        location = "Not available"
        no_of_employees = "Not available"
        services = "Not available"
        min_project_size = "Not available"



        # Extract data and replace default values if data is found
        agency_name = soup.find("a", {"class":"profile-information__company-title profile-information__company-link--tracking"})
        if agency_name:
            agency_name=agency_name.text.strip()
        
        website_link=soup.find('a', class_='profile-information__company-link profile-information__company-link--tracking')
        if website_link:
            # Extract the text content within the anchor tag
            link_text = website_link.text.strip()

            # Extract the href attribute value
            link_url = website_link.get('href')  # Get the link URL with the 'href' attribute

        services = soup.find_all("div", {"class":"client__industry"})
        service = [service.text.strip() for service in services]
        # Join the text with commas
        services = ', '.join(service)




        agency_size = soup.find('div', class_='profile-information__item-icon employees custom_popover').next_sibling
        if agency_size:
            no_of_employees=agency_size.text.strip()
        

        agency_location = soup.find("div", {"data-content":"<i>Company Size</i>"})
        if agency_location :
            location=agency_location .text.strip()

        project_size= soup.find("span", {"class":"profile-information__property-text"})
        if project_size :
            min_project_size=project_size.text.strip()
        
        description= soup.find("div", {"class":"profile-information__description profile-information__description--bottom"})
        if description :
            date_started=description.text.strip()
    


            # Create dictionary and append to list
        agencies = {
            "agency_name": agency_name,
            'link_text':link_text,
            'link_url':link_url,
            "website_link": website_link,
            "location":location,
            "date_started": date_started,
            "no_of_employees": no_of_employees,
            "min_project_size": min_project_size,
            "services": services
        }
        agency_list.append(agencies)
        print("agency_name:", agencies['agency_name'])


# # Stop the VPN thread if scraping is completed
stop_event.set()  # Signal the VPN thread to stop
vpn_thread.join()

df=pd.DataFrame(agency_list)

In [47]:
df.head(5)

,agency_name,link_text,link_url,website_link,location,date_started,no_of_employees,min_project_size,services
0,SmartSites,visit site,https://www.smartsites.com/lp/digital-marketin...,[\n visit site\n ],,SmartSites is a full-service digital agency lo...,250 - 999\n \n employees,"$1,000+","eCommerce\n 20%, Automotive\n 15%, D..."
1,SmartSites,visit site,https://www.smartsites.com/lp/digital-marketin...,[\n visit site\n ],,SmartSites is a full-service digital agency lo...,250 - 999\n \n employees,"$1,000+","eCommerce\n 20%, Automotive\n 15%, D..."
2,CHARLESON®,visit site,https://charleson.co.ke,[\n visit site\n ],,Charleson Group is a digital marketing firm ba...,10 - 49\n \n employees,"$1,000+","eCommerce\n 20%, Information technology\n..."
3,CHARLESON®,visit site,https://charleson.co.ke,[\n visit site\n ],,Charleson Group is a digital marketing firm ba...,10 - 49\n \n employees,"$1,000+","eCommerce\n 20%, Information technology\n..."
4,CHARLESON®,visit site,https://charleson.co.ke,[\n visit site\n ],,Charleson Group is a digital marketing firm ba...,10 - 49\n \n employees,"$1,000+","eCommerce\n 20%, Information technology\n..."


In [ ]:
import random
import subprocess
import threading
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

servers = ['Algeria', 'Egypt', 'Ghana', "Argentina", "Bahamas", "Bermuda", "Bolivia", "Brazil", "Brazil - 2", 'Kenya', 'Israel', 'Morocco', 'Lebanon',
           'South Africa', 'Albania', 'Andorra', 'Armenia']

def rotate_vpn(servers):
    while True:
        try:
            random_server = random.choice(servers)
            subprocess.run(['expressvpn', 'disconnect'], check=True)
            subprocess.run(['expressvpn', 'connect', random_server], check=True)
            print(f"Connected to {random_server}")
            time.sleep(5)  # Give the VPN time to stabilize
        except subprocess.CalledProcessError as e:
            print(f"Failed to switch VPN server: {e}")
        except KeyboardInterrupt:
            print("Stopping VPN rotation.")
            break
        time.sleep(500) 

# Start the VPN rotation in a separate thread
stop_event = threading.Event()
vpn_thread = threading.Thread(target=rotate_vpn, args=(servers,))
vpn_thread.start()

baseurl = "https://themanifest.com"
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.6533.103 Mobile Safari/537.36",
    "Mozilla/5.0 (iPod; CPU iPhone OS 17_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/127.0.6533.107 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (iPad; CPU OS 17_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/127.0.6533.107 Mobile/15E148 Safari/604.1"
    # Add more user agents here
]
user_agent = random.choice(USER_AGENTS)
headers = {
    'User-Agent': user_agent
}
agencylinks = []

for x in range(6):
    url = f"https://themanifest.com/ke/advertising/agencies?page={x}"
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.content, "lxml")

    # Find all <a> tags with the class "provider-card__profile-link"
    profile_links = soup.find_all('a', class_='provider-card__profile-link')
    
    # Extract the href attributes and concatenate with baseurl
    urls = [baseurl + link['href'] for link in profile_links]
    
    # Store the concatenated URLs in agencylinks
    agencylinks.extend(urls)


# Code to get all the data from each link
agency_list = []

for link in agencylinks:
    r = requests.get(link, headers=headers)
    soup = BeautifulSoup(r.content, "lxml")
    
    # Initialize variables with default values
    agency_name = "Not available"
    website_link = "Not available"
    date_started = "Not available"
    location = "Not available"
    no_of_employees = "Not available"
    services = "Not available"
    min_project_size = "Not available"

    # Extract data and replace default values if data is found
    agency_name_tag = soup.find("a", {"class":"profile-information__company-title profile-information__company-link--tracking"})
    if agency_name_tag:
        agency_name = agency_name_tag.text.strip()

    # Check if the agency is already in the list
    if any(agency['agency_name'] == agency_name for agency in agency_list):
        print(f"Skipping {agency_name} as it already exists in the list.")
        continue
    
    website_link_tag = soup.find('a', class_='profile-information__company-link profile-information__company-link--tracking')
    if website_link_tag:
        link_text = website_link_tag.text.strip()
        link_url = website_link_tag.get('href')

    services_tags = soup.find_all("div", {"class":"client__industry"})
    service = [service.text.strip() for service in services_tags]
    services = ', '.join(service)

    agency_size_tag = soup.find('div', class_='profile-information__item-icon employees custom_popover')
    if agency_size_tag:
        next_sibling = agency_size_tag.next_sibling
        if next_sibling:
            no_of_employees = next_sibling.text.strip()
        else:
            no_of_employees = None
    else:
        no_of_employees = None

    agency_location_tag = soup.find("span", {"class":"locality"})
    if agency_location_tag:
        location = agency_location_tag.text.strip()

    project_size_tag = soup.find("span", {"class":"profile-information__property-text"})
    if project_size_tag:
        min_project_size = project_size_tag.text.strip()

    description_tag = soup.find("div", {"class":"profile-information__description profile-information__description--bottom"})
    if description_tag:
        date_started = description_tag.text.strip()

    # Create dictionary and append to list
    agencies = {
        "agency_name": agency_name,
        'link_text': link_text,
        'link_url': link_url,
        "website_link": website_link_tag,
        "location": location,
        "date_started": date_started,
        "no_of_employees": no_of_employees,
        "min_project_size": min_project_size,
        "services": services
    }
    agency_list.append(agencies)
    print(f"Added {agency_name} to the list.")

# Stop the VPN thread if scraping is completed
stop_event.set()  # Signal the VPN thread to stop
vpn_thread.join()

df = pd.DataFrame(agency_list)

In [61]:
df.head()

,agency_name,link_text,link_url,website_link,location,date_started,no_of_employees,min_project_size,services
0,SmartSites,visit site,https://www.smartsites.com/lp/digital-marketin...,[\n visit site\n ],"Paramus, NJ",SmartSites is a full-service digital agency lo...,250 - 999\n \n employees,"$1,000+","eCommerce\n 20%, Automotive\n 15%, D..."
1,CHARLESON®,visit site,https://charleson.co.ke,[\n visit site\n ],"Nairobi, Kenya",Charleson Group is a digital marketing firm ba...,10 - 49\n \n employees,"$1,000+","eCommerce\n 20%, Information technology\n..."
2,Smith Aegis Plc - Marketing Agency Kenya,visit site,https://www.smithcorp.co.ke,[\n visit site\n ],"Nairobi, Kenya",Smith Aegis Plc is a marketing firm founded in...,10 - 49\n \n employees,"$1,000+","Advertising & marketing\n 10%, Media\n ..."
3,All Seasons Communications Limited,visit site,http://allseasonscommunications.co.ke,[\n visit site\n ],"Nairobi, Kenya",All Seasons Communications Limited is a digita...,2 - 9\n \n employees,Undisclosed,"Advertising & marketing\n 90%, eCommerce\..."
4,"H&S Reliance Group Ltd- Web Design, Mobile App...",visit site,https://www.hsreliancegroup.com/,[\n visit site\n ],"Nairobi, Kenya","H&S Reliance Group Ltd- Web Design, Mobile App...",10 - 49\n \n employees,"$1,000+",Advertising & marketing\n 100%


In [1]:
import os

In [4]:
os.chdir('C:/Users/Laptop')

In [62]:
df.to_csv('agencies.csv')